<a href="https://colab.research.google.com/github/wangy319/ROB535-Tean13-Final-Project/blob/main/Vehicle_Classification_Team_13_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vehicle Classification



## Setup Code
Before getting started, we need to run some boilerplate code to set up our environment, same as Assignment 1. You'll need to rerun this setup code each time you start the notebook.

First, run this cell load the autoreload extension. This allows us to edit .py source files, and re-import them into the notebook for a seamless editing and debugging experience.

In [ ]:
%load_ext autoreload
%autoreload 2

### Google Colab Setup
Next we need to run a few commands to set up our environment on Google Colab. If you are running this notebook on a local machine you can skip this section.

Run the following cell to mount your Google Drive. Follow the link, sign in to your Google account (the same account you used to store this notebook!) and copy the authorization code into the text box that appears below.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# TODO: Fill in the Google Drive path where you uploaded the assignment
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'ROB 535: Self Driving Car Final Project/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'Shared drives', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['dataset', 'perception', 'result', 'Meeting Agenda.gdoc', 'Controls ']


## Transfer Learning and Fine Tuning of ResNet architecture.

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.utils as utils
import cv2
import sys, os, time, copy

import numpy as np
import random
import csv
from tqdm import tqdm

sys.path.append(GOOGLE_DRIVE_PATH)

os.environ["TZ"] = "US/Eastern"
time.tzset()

# random seed
seed = 2021
random.seed(seed)
torch.manual_seed(seed)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


### Dataset


In [ ]:
class VehicleDataset(torch.utils.data.Dataset):

    def __init__(self, root, train=True, image_height=224): # TODO: adjust image size
        
        self.root = root
        self.train = train
        self.image_height = image_height;

        self.test_path = os.path.join(self.root, "test")
        self.train_val_path = os.path.join(self.root, "trainval")
        self.image_transform = transforms.Compose([
                        # transforms.ToPILImage(),
                        transforms.Resize(self.image_height),
                        # transforms.Grayscale(),
                        transforms.ToTensor()
                        # transforms.Normalize(  # TODO: calculate this
                        # 	mean=[0.4], # statistics calculated over 7scenes training set, should generalize fairly well
                        # 	std=[0.25]
                        # 	)
                        ])
        
        if train:
            # labels
            labels_path = os.path.join(self.train_val_path, "labels.csv")
            labels = []   # 0, 1, 2
            self.ids = []   # 'ee2d1ee9-dd1d-4e63-9881-19205c5dae27/0001'
            with open(labels_path, 'r') as csvfile:
                reader = csv.reader(csvfile, skipinitialspace=True)
                next(reader)
                for row in reader:
                    labels.append(int(row[1]))
                    self.ids.append(row[0])
            
            self.labels = np.zeros((len(labels), 3))
            for i, label in enumerate(labels):
                self.labels[i][label] = 1
            
        else: # test
            next(os.walk(self.test_path))
            self.ids = []
            for root, dirs, files in os.walk(self.test_path):
                path = root.split(os.sep)[-1]
                # print(path)
                # print(files)
                img_ids = [f[:4] for f in files if f.endswith('.jpg')]
                file_list = [os.path.join(path, id) for id in img_ids]
                self.ids.extend(file_list)
            # print(self.ids)
            # print(len(self.ids))
            
        # self.bboxes = np.loadtxt(os.path.join(root, "boxes.txt")).reshape(60, 17, 4)   # (60, 17, 4)
        # self.labels = np.loadtxt(os.path.join(root, "labels.txt"), dtype=np.int64)    # (60, 17) (1s, 2s) 1-crop, 2-weed


    def __getitem__(self, idx):
        if self.train:
            # load images
            img_path = os.path.join(self.train_val_path, self.ids[idx]+"_image.jpg")
            img = Image.open(img_path).convert("RGB") # w*h*3
            img = self.image_transform(img)
            labelarray = np.array([self.labels[idx]])
            label = torch.tensor(labelarray)

            return img, label
        else:
            img_path = os.path.join(self.test_path, self.ids[idx]+"_image.jpg")
            img = Image.open(img_path).convert("RGB") # w*h*3
            img = self.image_transform(img)
            return img, self.ids[idx]

    def __len__(self):
        return len(self.ids)

### Model Definition

Class Resnet18 is a modified version of the original Resnet18 architecture. 


In [ ]:
class Resnet18(nn.Module):
    def __init__(self, num_classes=3, use_pretrained=True):
        super(Resnet18, self).__init__()
        self.model = models.resnet18(pretrained=use_pretrained)
        # self.model.avgpool = None
        num_ftrs = self.model.fc.in_features
        self.model.fc = torch.nn.Linear(num_ftrs, 256)
        self.classifier_layer = nn.Sequential(
            # nn.Linear(512, 256), # use the original linear layer on ResNet18
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.2),
            nn.Linear(256, 128), 
            nn.ReLU(), # batchnorm after activation yields better result
            nn.BatchNorm1d(128),
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
      batch_size ,_,_,_ = x.shape     #taking out batch_size from input image
      x = self.model(x)

      x = self.classifier_layer(x)
      return x

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes=3, feature_extract=False, use_pretrained=True, input_size=224):
    if model_name == 'resnet18':
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = torch.nn.Linear(num_ftrs, num_classes)
        input_size = input_size

    elif model_name == "resnet18_modified":
        model_ft = Resnet18(num_classes)
        
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size


### Training


In [ ]:
def train_model(model, dataloaders, loss_func, optimizer, num_epochs=25, is_inception=False, result_path="./"):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        # for phase in ['train', 'val']:
        for phase in ['train']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)

                    labels = labels.reshape(-1, 3)

                    loss = loss_func(outputs, labels)

                    _, preds = torch.max(outputs, 1)
                    _, labels_max = torch.max(labels, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels_max.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
        
            # save every model
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, os.path.join(result_path, str(epoch)))

            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    # print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
# Configs
image_height = 512
# image_height = 32
num_classes = 3
TRAIN = True
num_epochs = 100
batch_size = 32

print('Using device:', device)
print()

data_path = os.path.join(GOOGLE_DRIVE_PATH, "dataset/" + "rob535-fall2021-final-project-data")

# Dataset initalization
dataset = VehicleDataset(data_path, train=TRAIN, image_height=image_height) # 7574
split_train_val = round(len(dataset)*0.9)
train_set, val_set = torch.utils.data.random_split(dataset, [split_train_val, len(dataset)-split_train_val])
train_loader = torch.utils.data.DataLoader(
                            train_set, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(
                            val_set, batch_size=batch_size, shuffle=True, num_workers=0)
data_loaders = {}
data_loaders['train'] = train_loader;
data_loaders['val'] = val_loader;

# Model Initialization - original/modified resnet18 architecture
model,_ = initialize_model(model_name="resnet18_modified", feature_extract=False, num_classes=num_classes, input_size=image_height)
# model,_ = initialize_model(model_name="resnet18", feature_extract=False, num_classes=num_classes, input_size=image_height)

# print(model) # uncomment this line to view model architecture
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0005, 
                             betas=(0.9, 0.999), eps=1e-08, weight_decay=0.000005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

# Setup the loss fxn
loss_func = torch.nn.CrossEntropyLoss()

# Train and evaluate
result_path = os.path.join(GOOGLE_DRIVE_PATH, "result")
model_ft, hist = train_model(model, data_loaders, loss_func, optimizer, num_epochs=num_epochs, result_path=result_path)

Run the following cell to visualize model architecture - Original ResNet18

In [ ]:
model,_ = initialize_model(model_name="resnet18", feature_extract=False, num_classes=num_classes, input_size=image_height)
print(model)

Run the following cell to visualize model architecture - Modified ResNet18

In [ ]:
model,_ = initialize_model(model_name="resnet18_modified", feature_extract=False, num_classes=num_classes, input_size=image_height)
print(model)

### Model Evaluation


In [ ]:
def evaluate_model(model, test_loader, result_path, save=False):
    model.eval()

    if save:
        # save predictions
        rows = []

        for inputs, id in tqdm(test_loader):
            # print(id)
            inputs = inputs.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(False):
                # Get model outputs and calculate loss
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

                rows.append([id[0], preds.item()])

        # name of csv file 
        filename = os.path.join(result_path, "13.csv")
        fields = ["guid/image", "label"]
        # writing to csv file 
        with open(filename, 'w') as csvfile: 
            # creating a csv writer object 
            csvwriter = csv.writer(csvfile) 
            # writing the fields 
            csvwriter.writerow(fields) 
            # writing the data rows 
            csvwriter.writerows(rows)

    else:
        for inputs, id in tqdm(test_loader):
            inputs = inputs.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            # track history if only in train
            with torch.set_grad_enabled(False):
                # Get model outputs and calculate loss
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                
                print("id ", id)
                print("preds ", preds.item())


Run the following cell to generate predictions for test set. 

Update "data_path", "result_path" and "evaluation_epoch" to desired saved model.

Setting $save$ to True will generate the .csv file and save in the "result_path".

```
evaluate_model(model, test_loader, result_path, save=True)
```

In [ ]:
# Configs
image_height = 512
# image_height = 32
num_classes = 3
TRAIN = False
EVAL = True
num_epochs = 100
batch_size = 64

print('Using device:', device)
print()

data_path = os.path.join(GOOGLE_DRIVE_PATH, "dataset/" + "rob535-fall2021-final-project-data")
result_path = os.path.join(GOOGLE_DRIVE_PATH, "result/"+"modified_weight_decay")
evaluation_epoch = "11"

# Dataset initalization
test_dataset = VehicleDataset(data_path, train=False, image_height=image_height) # 7574
test_loader = torch.utils.data.DataLoader(
                            test_dataset, batch_size=1, shuffle=False, num_workers=0)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0005, 
                             betas=(0.9, 0.999), eps=1e-08, weight_decay=0.000005)

# Model Initialization
model,_ = initialize_model(model_name="resnet18_modified", feature_extract=False, num_classes=num_classes, input_size=image_height)
model.load_state_dict(torch.load(os.path.join(result_path, "1"), map_location=torch.device('cpu'))) #  map_location=torch.device('cpu') can be commented out if using gpu
model.to(device)

# print("len(test_loader)", len(test_loader))

evaluate_model(model, test_loader, result_path, save=True)

